In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
%%writefile multiply.cu
#include <iostream>
#include <cuda_runtime.h>

// Kernel for matrix multiplication
__global__ void matmul(int* A, int* B, int* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < N && col < N) {
        int sum = 0;
        for (int k = 0; k < N; k++) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

void matrixMultiplication() {
    int N = 512;
    int size = N * N * sizeof(int);
    int *A, *B, *C, *d_A, *d_B, *d_C;

    cudaMallocHost(&A, size);
    cudaMallocHost(&B, size);
    cudaMallocHost(&C, size);
    cudaMalloc(&d_A, size);
    cudaMalloc(&d_B, size);
    cudaMalloc(&d_C, size);

    // Initialize matrices with smaller values to avoid overflow
    for (int i = 0; i < N * N; i++) {
        A[i] = (i % 10) + 1;
        B[i] = (i % 10) + 1;
    }

    cudaMemcpy(d_A, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, size, cudaMemcpyHostToDevice);

    dim3 blockSize(16, 16);
    dim3 gridSize((N + 15) / 16, (N + 15) / 16);
    matmul<<<gridSize, blockSize>>>(d_A, d_B, d_C, N);
    cudaDeviceSynchronize();

    cudaMemcpy(C, d_C, size, cudaMemcpyDeviceToHost);

    std::cout << "Matrix Multiplication Result (First 5x5 values):\n";
    for (int i = 0; i < 5; i++) {
        for (int j = 0; j < 5; j++) {
            std::cout << C[i * N + j] << " ";
        }
        std::cout << "\n";
    }
    std::cout << std::flush;

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    cudaFreeHost(A); cudaFreeHost(B); cudaFreeHost(C);
}

int main() {
    matrixMultiplication();
    return 0;
}

Writing multiply.cu


In [3]:
!nvcc -arch=sm_75 multiply.cu -o multiply

In [4]:
!./multiply

Matrix Multiplication Result (First 5x5 values):
16072 18880 14038 16846 13024 
13020 15832 14054 16866 16108 
14048 16864 13050 15866 13072 
14056 16876 16126 18946 14116 
13044 15868 13082 15906 14140 
